## Limpieza de datos

- Nicolas Silva
- Rosa Huamaní

1. Cargamos la data y configuramos

In [112]:
import pandas as pd
import os
import pyreadstat

# Configuración del directorio y rutas de los archivos
current_dir = os.getcwd()
data_folder = os.path.join(current_dir, "Elecciones")
data_path1 = os.path.join(data_folder, "elecciones2010distb.dta")
data_path2 = os.path.join(data_folder, "elecciones2014distb.dta")
data_path3 = os.path.join(data_folder, "elecciones2018distb.dta")
data_path4 = os.path.join(data_folder, "ERM2022_Municipal_Distrital.csv")

# Leer archivos Stata (.dta) y extraer solo los DataFrames
d10, meta10 = pyreadstat.read_dta(data_path1)
d14, meta14 = pyreadstat.read_dta(data_path2)
d18, meta18 = pyreadstat.read_dta(data_path3)

# Leer archivo CSV con configuración adecuada
d22 = pd.read_csv(
    data_path4,
    sep=";",  # Cambia esto si el separador no es ";"
    encoding="utf-8",  # Cambia la codificación si es necesario
    on_bad_lines="skip",  # Ignora líneas mal formateadas
    engine="python"  # Motor más flexible para leer archivos complejos
)


In [113]:
d10["Año"] = 2010 #Añadimos columna año
d14["Año"] = 2014
d18["Año"] = 2018
d22["Año"] = 2022

In [114]:
print(d10.columns)

Index(['Eleccin', 'Tipo_Eleccin', 'Regin', 'Provincia', 'Distrito',
       'Electores', 'Organizacin_Poltica', 'Votos', 'Ubigeo', 'Votosemitidos',
       'Participación', 'Votosválidos', 'Votos_porc', 'Votos_ganador_porc',
       'Votos_segundo_porc', 'margvictoria', 'N_Partidos', 'Año'],
      dtype='object')


In [115]:
print(d14.columns)


Index(['Región', 'Provincia', 'Distrito', 'Electores', 'Participación',
       'Votosemitidos', 'OrganizaciónPolítica', 'TipoOrganizaciónPolítica',
       'Votos', 'Ubigeo', 'Votosválidos', 'Votos_porc', 'Votos_ganador_porc',
       'Votos_segundo_porc', 'margvictoria', 'N_Partidos', 'Año'],
      dtype='object')


In [116]:
print(d18.columns)

Index(['Region', 'Provincia', 'Distrito', 'Electores', 'Participación',
       'Votosemitidos', 'OrganizaciónPolítica', 'TipoOrganizaciónPolítica',
       'Votos', 'Ubigeo', 'Votosválidos', 'Votos_porc', 'Votos_ganador_porc',
       'Votos_segundo_porc', 'margvictoria', 'N_Partidos', 'Año'],
      dtype='object')


Región, Provincia, Distrito, Número de partidos, Porcentaje de Ausentismo, Porcentaje de participación

In [117]:
import pandas as pd

def process_election_data(df, year, rename_region=None):
    """
    Procesa una base de datos electoral, corrigiendo el cálculo de P_ausentismo.
    
    Args:
    - df: DataFrame con los datos originales.
    - year: Año de la elección (para añadirlo como columna).
    - rename_region: Nombre actual de la columna 'Region' a renombrar.
    
    Returns:
    - DataFrame procesado.
    """
    if rename_region:
        df = df.rename(columns={rename_region: "Region"})
    
    df['P_ausentismo'] = (df['Electores'] - df['Votosemitidos']) / df['Electores']
    df['P_ausentismo'] = df['P_ausentismo'].clip(lower=0, upper=1)  # Limitar a [0, 1]

    selected_columns = ["Año","Region", "Provincia", "Distrito", "Electores", "Votosemitidos","Ubigeo", "P_ausentismo", "N_Partidos"]
    df = df[selected_columns]
    
    
    return df

d10_processed = process_election_data(d10, 2010, rename_region="Regin")
d14_processed = process_election_data(d14, 2014, rename_region="Región")
d18_processed = process_election_data(d18, 2018)  # Sin renombrar

print("d10 procesado:")
print(d10_processed.head())

print("\nd14 procesado:")
print(d14_processed.head())

print("\nd18 procesado:")
print(d18_processed.head())


d10 procesado:
    Año    Region    Provincia  Distrito  Electores  Votosemitidos  Ubigeo  \
0  2010  AMAZONAS  CHACHAPOYAS  ASUNCION        234          211.0  010102   
1  2010  AMAZONAS  CHACHAPOYAS  ASUNCION        234          211.0  010102   
2  2010  AMAZONAS  CHACHAPOYAS  ASUNCION        234          211.0  010102   
3  2010  AMAZONAS  CHACHAPOYAS  ASUNCION        234          211.0  010102   
4  2010  AMAZONAS  CHACHAPOYAS  ASUNCION        234          211.0  010102   

   P_ausentismo  N_Partidos  
0      0.098291         3.0  
1      0.098291         3.0  
2      0.098291         3.0  
3      0.098291         3.0  
4      0.098291         3.0  

d14 procesado:
    Año    Region    Provincia  Distrito  Electores  Votosemitidos  Ubigeo  \
0  2014  AMAZONAS  CHACHAPOYAS  ASUNCION        298            280  010102   
1  2014  AMAZONAS  CHACHAPOYAS  ASUNCION        298            280  010102   
2  2014  AMAZONAS  CHACHAPOYAS  ASUNCION        298            280  010102   
3  2014 

Consolidamos el merge de las tres datas. Faltaría de la última

In [118]:
elecciones_loc = pd.concat([d10_processed, d14_processed, d18_processed], ignore_index=True)

Revisamos las variables y cómo se llaman en estas tres datas para transformar la data del 2022

In [119]:
print(elecciones_loc.columns)

Index(['Año', 'Region', 'Provincia', 'Distrito', 'Electores', 'Votosemitidos',
       'Ubigeo', 'P_ausentismo', 'N_Partidos'],
      dtype='object')


In [120]:
print(d22.columns)

Index(['UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'TIPO_ELECCION',
       'MESA', 'ESTADO_MESA', 'UBICACION_EN_CEDULA', 'TIPO_AGRUPACION',
       'CODIGO_AGRUPACION', 'AGRUPACION_POLITICA', 'VOTOS_OBTENIDOS',
       'ELECTORES_HABILES', 'VOTOS_BLANCOS', 'VOTOS_NULOS', 'VOTOS_IMPUG',
       'Año'],
      dtype='object')


In [121]:
col_drop =["VOTOS_NULOS","VOTOS_BLANCOS","VOTOS_IMPUG","CODIGO_AGRUPACION","UBICACION_EN_CEDULA","ESTADO_MESA","MESA","TIPO_ELECCION"]

d22 = d22.drop(columns=col_drop)

In [122]:
print(d22.columns)

Index(['UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'TIPO_AGRUPACION',
       'AGRUPACION_POLITICA', 'VOTOS_OBTENIDOS', 'ELECTORES_HABILES', 'Año'],
      dtype='object')


In [123]:
d22.columns = ["Ubigeo","Region","Provincia","Distrito","Tipo_agrupacion","Agrupacion","Votos","Electores","Año"]

In [126]:
# Agregar la columna de Año al DataFrame
d22["Año"] = 2022  # Asegúrate de asignar el año correspondiente

# Crear la columna de partidos/movimientos
d22["Es_Partido_Movimiento"] = d22["Tipo_agrupacion"].str.contains(
    "PARTIDO POLÍTICO|MOVIMIENTO REGIONAL", case=False, regex=True
)

# Calcular el número de partidos/movimientos únicos por distrito
partidos_movimientos_por_distrito = (
    d22[d22["Es_Partido_Movimiento"]]
    .groupby(["Region", "Provincia", "Distrito", "Ubigeo"])["Agrupacion"]
    .nunique()
    .reset_index(name="N_Partidos")  
)

# Calcular el total de votos emitidos por distrito
total_votos_por_distrito = (
    d22.groupby(["Region", "Provincia", "Distrito", "Ubigeo"])["Votos"]
    .sum()
    .reset_index(name="Votosemitidos")
)

# Calcular la proporción de votos respecto a electores
d22["Proporcion_Votos_Electores"] = d22["Votos"] / d22["Electores"]

# Calcular la proporción promedio por distrito
proporcion_por_distrito = (
    d22.groupby(["Region", "Provincia", "Distrito", "Ubigeo"])["Proporcion_Votos_Electores"]
    .mean()
    .reset_index(name="P_ausentismo")
)

# Unir los resultados al DataFrame original
d22_resultado = (
    d22.merge(partidos_movimientos_por_distrito, on=["Region", "Provincia", "Distrito", "Ubigeo"], how="left")
       .merge(total_votos_por_distrito, on=["Region", "Provincia", "Distrito", "Ubigeo"], how="left")
       .merge(proporcion_por_distrito, on=["Region", "Provincia", "Distrito", "Ubigeo"], how="left")
)

# Filtrar solo las columnas relevantes, incluyendo `Año`
d22_resultado = d22_resultado[
    ["Region", "Provincia", "Distrito", "Ubigeo", "Votosemitidos", "Electores", "N_Partidos", "P_ausentismo", "Año"]
].drop_duplicates()

# Mostrar el resultado
print(d22_resultado.head())


      Region    Provincia  Distrito  Ubigeo  Votosemitidos  Electores  \
0   AMAZONAS  CHACHAPOYAS  ASUNCION   10102          267.0      170.0   
4   AMAZONAS  CHACHAPOYAS    BALSAS   10103          538.0      264.0   
8   AMAZONAS  CHACHAPOYAS    BALSAS   10103          538.0      265.0   
20  AMAZONAS  CHACHAPOYAS     CHETO   10104          514.0      214.0   
23  AMAZONAS  CHACHAPOYAS     CHETO   10104          514.0      215.0   

    N_Partidos  P_ausentismo   Año  
0            2      0.392647  2022  
4            4      0.127044  2022  
8            4      0.127044  2022  
20           3      0.266046  2022  
23           3      0.266046  2022  


In [127]:
# Unir los DataFrames
data_combinada = pd.concat([d22_resultado, elecciones_loc], ignore_index=True)

# Verificar el resultado
print(data_combinada.head())
print(data_combinada.info())  # Verificar número de filas y columnas


     Region    Provincia  Distrito Ubigeo  Votosemitidos  Electores  \
0  AMAZONAS  CHACHAPOYAS  ASUNCION  10102          267.0      170.0   
1  AMAZONAS  CHACHAPOYAS    BALSAS  10103          538.0      264.0   
2  AMAZONAS  CHACHAPOYAS    BALSAS  10103          538.0      265.0   
3  AMAZONAS  CHACHAPOYAS     CHETO  10104          514.0      214.0   
4  AMAZONAS  CHACHAPOYAS     CHETO  10104          514.0      215.0   

   N_Partidos  P_ausentismo   Año  
0         2.0      0.392647  2022  
1         4.0      0.127044  2022  
2         4.0      0.127044  2022  
3         3.0      0.266046  2022  
4         3.0      0.266046  2022  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50818 entries, 0 to 50817
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Region         50818 non-null  object 
 1   Provincia      50818 non-null  object 
 2   Distrito       50818 non-null  object 
 3   Ubigeo         50818 non

In [128]:
data_combinada.dropna()

,Region,Provincia,Distrito,Ubigeo,Votosemitidos,Electores,N_Partidos,P_ausentismo,Año
0,AMAZONAS,CHACHAPOYAS,ASUNCION,10102,267.0,170.0,2.0,0.392647,2022
1,AMAZONAS,CHACHAPOYAS,BALSAS,10103,538.0,264.0,4.0,0.127044,2022
2,AMAZONAS,CHACHAPOYAS,BALSAS,10103,538.0,265.0,4.0,0.127044,2022
3,AMAZONAS,CHACHAPOYAS,CHETO,10104,514.0,214.0,3.0,0.266046,2022
4,AMAZONAS,CHACHAPOYAS,CHETO,10104,514.0,215.0,3.0,0.266046,2022
...,...,...,...,...,...,...,...,...,...
50813,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305,2429.0,2837.0,10.0,0.143814,2018
50814,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305,2429.0,2837.0,10.0,0.143814,2018
50815,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305,2429.0,2837.0,10.0,0.143814,2018
50816,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305,2429.0,2837.0,10.0,0.143814,2018


In [129]:
data_combinada.to_csv("elecciones_limpio.csv", index=False, encoding ="utf-8")